In [1]:
import vamp
import librosa
import numpy as np
import pretty_midi
import jams
import os
import glob

In [2]:
def rough_midi(inpath, outpath=None):
    # gets a rough midi trasncription using pYin Note from inpath.wav to outpath.mid
    if os.path.exists(outpath):
        print('file already exist')
        return 0
        #check if outpath exist, return 0.
        
    y, fs = librosa.load(inpath, sr=None)
#     tempo = int(inpath.split('_')[1].split('-')[1])
    print('finished loading')
    
    param = {"threshdistr": 2,
             "lowampsuppression": 0.08,
             "outputunvoiced": 2,
             "precisetime": 0,
             "prunethresh": 0.05,
             "onsetsensitivity": 0.8}

    pyin_note_output = vamp.collect(y, fs, 'pyin:pyin', output='notes', parameters=param)['list']
    print('finished pYin')
    midi = build_midi_from_output(pyin_note_output)
    
    if outpath:
        midi.write(outpath)
        return 0
    
    return midi


def build_midi_from_output(pyin_note_output):
    midi = pretty_midi.PrettyMIDI('tempo_template.mid')
    ch = pretty_midi.Instrument(program=25)
    for note in pyin_note_output:
        pitch = int(round(librosa.hz_to_midi(note['values'])[0]))
        st = float(note['timestamp'])
        dur = float(note['duration'])
#         print(pitch, st, dur )
        n = pretty_midi.Note(
            velocity=100,
            pitch=pitch, start=st,
            end=st+dur
        )

        ch.notes.append(n)
#         bend_amount = int(round((note.value - pitch) * 4096))
#         pb = pretty_midi.PitchBend(pitch=bend_amount*q, time=st)
#         ch.pitch_bends.append(pb)
    midi.instruments.append(ch)
    return midi

In [3]:
fnames = glob.glob('./audio_stem/*.wav')
print(fnames)

['./audio_stem/3_c_1.wav', './audio_stem/2_c_1.wav', './audio_stem/0_c_1.wav', './audio_stem/1_c_1.wav', './audio_stem/4_c_1.wav', './audio_stem/5_c_1.wav']


In [4]:
for i, fname in enumerate(fnames):
    basename_trimed = os.path.basename(fname).split('.')[0]
    rough_midi('./audio_stem/' + basename_trimed + '.wav', './onset_midi_est/' + basename_trimed + '.mid')
    print(i+1, len(fnames))

file already exist
1 6
file already exist
2 6
file already exist
3 6
finished loading
finished pYin
4 6
finished loading
finished pYin
5 6
finished loading
finished pYin
6 6
